In [1]:
#%pip install beautifulsoup4
#%pip install pip-system-certs

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import os
from langchain_community.graphs import Neo4jGraph
from PyPDF2 import PdfReader
from langdetect import detect
import spacy
import re
from unidecode import unidecode
import time

Conectando ao Neo4j 

In [2]:
from neo4j import GraphDatabase

# Neo4j variables
NEO4J_URL = os.getenv("NEO4J_URL")
NEO4J_USERNAME =os.getenv("NEO4J_USERNAME")
NEO4J_PASSWORD = os.getenv("NEO4J_PASSWORD")


graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

- Buscar os links para a tese no Neo4j
- Buscar o link do PDF no repositório da universidade
- Baixar o PDF
- Extrair o texto do PDF 

Buscar links para a tese no Neo4j

In [10]:
# Query to get theses and their repositories from the Neo4j database
# This query retrieves theses that are published by the Federal University of Santa Catarina (UFSC) and have not been processed yet.
query1 = """
MATCH (t:Thesis)-[:publisher]-(uni:University{uri:'tag:stardog:api:UFSC'})
WHERE NOT EXISTS {(t{processed:true})}
RETURN distinct(t["uri"]) as Thesis, t["repository"] as repository 
"""

# Alternative query to get theses that do not have excerpts and have been processed.
query2 = """ 
match (t:Thesis)-[:publisher]-(uni:University{uri:'tag:stardog:api:UFSC'})
where (not (t:Thesis)-[:BFO_0000050]-(:Exerpt)) and  exists {(t{processed:true})}
return distinct(t["uri"]) as Thesis, t["repository"] as repository
"""

# Query to retrive theses with few pages and their repositories.
query3 = """
MATCH (t:Thesis)-[:publisher]-(uni:University{uri:'tag:stardog:api:UFSC'})
Match (t:Thesis)-[:BFO_0000050]-(e:Exerpt)
RETURN count(e) as pages, t["uri"] as Thesis, t["repository"] as repository
 order by (pages) limit 40
"""

query4 = """
MATCH (t:Thesis)-[:publisher]-(uni:University{uri:'tag:stardog:api:UFSC'})
WHERE NOT EXISTS {(t{processed:true})}
RETURN distinct(t["uri"]) as Thesis, t["title"] as title 
"""

query5 = """
MATCH (t:Thesis {processed: true})-[:publisher]-(uni:University {uri: 'tag:stardog:api:UFSC'})
WHERE NOT (t)--(:Exerpt)
RETURN distinct(t["uri"]) as Thesis, t["title"] as title 
"""

graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
results = graph.execute_query(query4).records
graph.close()

#results = graph.query(query2)
# Convert the results to a DataFrame
#thesis_repository = pd.DataFrame(results, columns=["pages", "Thesis", "repository"])
thesis_repository = pd.DataFrame(results, columns=["Thesis", "title"])
thesis_repository = thesis_repository[2:].reset_index(drop=True) 
thesis_repository

,Thesis,title
0,tag:stardog:api:imagens_invisiveis_de_africas_...,Imagens invisíveis de Áfricas presentes : expe...
1,tag:stardog:api:silencios_na_historia_e_disput...,Silêncios na História e disputas entre memória...
2,tag:stardog:api:proposta_de_acoes_para_ingress...,Proposta de ações para ingresso e permanência ...
3,tag:stardog:api:criminologia_e_racismo_,Criminologia e racismo :
4,tag:stardog:api:mulheres_negras_soropositivas_...,Mulheres negras soropositivas e as interseccio...
...,...,...
209,tag:stardog:api:jequitiboaba_do_samba_o_eco_da...,Jequitiboabá do samba: o eco da voz do dono
210,tag:stardog:api:avaliacao_da_evapotranspiracao...,Avaliação da evapotranspiração real da bacia d...
211,tag:stardog:api:a_protecao_integral_de_crianca...,A Proteção integral de crianças e adolescentes...
212,tag:stardog:api:ue_gaucho_em_floripa_tem_samba...,"Ué gaúcho, em Floripa tem samba?: uma antropol..."


Criando as utils functions

In [11]:
# Funcao para coletar link do repositorio na página da BDTD

def get_repository_link(title):

    bdtd_url = 'https://bdtd.ibict.br/vufind/Search/Results?lookfor=' + title.replace(' ', '+') + '&type=Title' #&limit=1'
    #Fazer requisição e parsear o arquivo html
    try:
        f = requests.get(bdtd_url, verify=True).text#, verify=False) 
    except:
        print("Sleeping for 5 seconds before retrying... (get_repository_link)")
        time.sleep(5)
        try:
            f = requests.get(bdtd_url, verify=True).text#, verify=False) 
        except:
            print("Sleeping for 30 seconds before retrying... (get_repository_link)")
            time.sleep(30)
            f = requests.get(bdtd_url, verify=True).text#, verify=False)    
    soup = bs(f, "html.parser")

    #Coletando link para o repositório
    link_list = []
    for doc in soup.find_all('a', href=True):

        if (doc['href'].startswith('https://repositorio.ufsc.br/') |
            doc['href'].startswith('http://repositorio.ufsc.br/')):
            path = doc['href']
            link_list.append(path)
    if link_list != []:
        return list(set(link_list))[0]
    else:
        return None



# Funcao para coletar link do pdf

def get_pdf_link(url):
    
    #Fazer requisição e parsear o arquivo html
    try:
        f = requests.get(url, verify=True).text#, verify=False) 
    except:
        print("Sleeping for 5 seconds before retrying... (get_pdf_link)")
        time.sleep(5)
        try:
            f = requests.get(url, verify=True).text#, verify=False) 
        except:
            print("Sleeping for 30 seconds before retrying... (get_pdf_link)")
            time.sleep(30)
            f = requests.get(url, verify=True).text#, verify=False)    
    soup = bs(f, "html.parser")
    
    #Coletando link para as teses
    link_list = []
    for doc in soup.find_all('a', href=True):
        if doc['href'].startswith('/bitstream'):     # (doc['href'].endswith('.pdf')):# & doc['href'].startswith('/bitstream')):
            path = doc['href']
            prefix_uni = 'https://repositorio.ufsc.br'
            link = prefix_uni + path
            link_list.append(link)
    if link_list != []:
        return list(set(link_list))
    else:
        return None
    
    #return None

#fazer download do arquivo
def download_file(pdf_link, thesis_uri):


    # NOTE the stream=True parameter below
    try:
        r = requests.get(pdf_link, verify=True, stream=True)
    except:
        print("Sleeping for 5 seconds before retrying... (download_file)")
        time.sleep(5)
        try:
            r = requests.get(pdf_link, verify=True, stream=True)
        except:
            print("Sleeping for 30 seconds before retrying... (download_file)")
            time.sleep(30)
            r = requests.get(pdf_link, verify=True, stream=True)
    # raise_for_status() is not needed with stream=True
    # r.raise_for_status()
    with open('tese UFSC.pdf', 'wb') as f:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
    return 

#Carregando os modelos SpaCy para inglês e português 
nlp_en = spacy.load("en_core_web_lg")
nlp_pt = spacy.load("pt_core_news_lg")

# Funcão que recebe uma string e a limpar para ficar no formato aceitável para uma URI
def process_uri(x):
    return (re.sub('[^a-zA-Z0-9_ ]', '',
            unidecode(x.strip())
                        .replace(" ", "_")
                        .replace("[","")
                        .replace("]","")
                        .replace("?","")
                        .replace("'","")
                        .lower()))


# Função para extrair texto de uma tese em PDF e identificar entidades
def extratc_text(tese_path):
    
    # Processando o arquivo em PDF
    try:
        reader = PdfReader(tese_path)
        number_of_pages = len(reader.pages)
    except:
        print(f"Erro ao ler o arquivo {tese_path}. ")
        excerpt = {}
        return excerpt
    
    excerpt = {}
    

    # Iterando por cada página
    for page_number in range(number_of_pages):
        
        
        
        # Extraindo o texto da página
        try:
            if page_number == 0:
                try:
                    page_text = (reader.pages[page_number].extract_text() + '\n ' 
                                + reader.pages[page_number+1].extract_text()[:400])
                except:
                    page_text = reader.pages[page_number].extract_text()
            else:
                
                if page_number == number_of_pages -1:
                    try:
                        page_text = (reader.pages[page_number-1].extract_text()[-400:] + '\n ' 
                                    + reader.pages[page_number].extract_text())
                    except:
                        page_text = reader.pages[page_number].extract_text()
                
                else:
                    try:
                        page_text = (reader.pages[page_number-1].extract_text()[-400:] + '\n ' 
                                    + reader.pages[page_number].extract_text() + '\n ' 
                                    + reader.pages[page_number+1].extract_text()[:400])  
                    except:
                        page_text = reader.pages[page_number].extract_text()
        except:
            page_text = ''
        
        # Limpando page text         
        page_text = page_text.replace("'", "").replace('"', '')
        # Verifica se o último caractere é uma barra invertida
        if page_text != '':
            if page_text[-1] == '\\': 
                page_text = page_text[:-2]
        page_text = re.sub(r"[\ud800-\udfff]", " ", page_text)
        page_text = page_text.encode('utf-8').decode('utf-8')
        
        
        # Detectando o idioma do texto
        try:
            lang = detect(page_text)
            
        except:
            lang = 'Não_detectado'
        
        
        # Extraindo as entidades
        page_extracted = False
        if lang == 'pt':
            doc = nlp_pt(page_text)
            page_extracted = True
        if lang == 'en':
            doc = nlp_en(page_text)
            page_extracted = True
        

        if page_extracted:

            persons = []
            gpes = []
            #orgs = []
            for ent in doc.ents:
                if ent.label_ == "PER": #"PERSON":
                    persons.append(process_uri(ent.text))
                if ent.label_ == "LOC": #"GPE":
                    gpes.append(process_uri(ent.text))
                #if ent.label_ == "ORG":
                #    orgs.append(process_uri(ent.text))
                    
            excerpt[page_number] = {'text': page_text, 
                                    'lang': lang, 
                                    'persons': list(set(persons)), 
                                    'gpes': list(set(gpes)),
                                #'orgs': list(set(orgs))
                                }

    return excerpt

Iterando por todas as URI para extrair o texto e gravar no grafo NEO4j

In [ ]:
# Gravar hora de início do processamento
start_time = pd.Timestamp.now()
parcial_time = pd.Timestamp.now()

for i in range(0,len(thesis_repository)):

    title = thesis_repository['title'][i].replace("?","").replace('-', ' ').replace('&', ' ').replace('/', ' ').replace('\\', ' ')  
    repo = get_repository_link(title)

    # Coletando o link do PDF
    #url = thesis_repository['repository'][i]
    thesis_uri = thesis_repository['Thesis'][i][16:]
    pdf_link = get_pdf_link(repo)
    if pdf_link is not None:
        # Itereando pelos links de PDF encontrados (em alguns casos há outros materiais além da tese)
        for pdf_num in range(len(pdf_link)):
            print(f"{i} - Downloading and extracting: {thesis_uri} from {pdf_link[pdf_num]}")
            download_file(pdf_link[pdf_num], thesis_uri)

            # Extraindo o texto e entidades
            tese_path = 'tese UFSC.pdf'
            excerpt = extratc_text(tese_path)

            excerpts = []
            persons = []
            gpes = []

            # Salvando as triplas em CSV
            for n in excerpt.keys():

                excerpt_uri = thesis_repository['Thesis'][i] + '_pdf_' + str(pdf_num) + '_p_' + str(n)
                excerpt_text = excerpt[n]['text']
                # Verifica se o último caractere é uma barra invertida
                #if excerpt_text[-1] == '\\': 
                #    excerpt_text = excerpt_text[:-1]
                

                excerpt_page = 'pdf_' + str(pdf_num) + '_p_' + str(n)
                excerpt_lang = excerpt[n]['lang']
                
                excerpts.append([thesis_repository['Thesis'][i], excerpt_uri, excerpt_text, excerpt_page, excerpt_lang])
                    
                #Criando as relações com as entidades
                for person in excerpt[n]['persons']:
                    persons.append([excerpt_uri, 'tag:stardog:api:' + person])
                
                for gpe in excerpt[n]['gpes']:
                    gpes.append([excerpt_uri, 'tag:stardog:api:' + gpe])
            
            # Salvando os dados em CSV no diretório de importação do Neo4j
            pd.DataFrame(excerpts).to_csv('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\excerptsUFSC.csv', index=False, header=False)
            pd.DataFrame(persons).to_csv('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\personsUFSC.csv', index=False, header=False)
            pd.DataFrame(gpes).to_csv('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\gpesUFSC.csv', index=False, header=False)

            #Deletando o arquivo PDF após o processamento
            os.remove('tese UFSC.pdf')

            print(f"Loading triplas: {thesis_uri}")

            # Importanto os dados para o Neo4j
            #Excerpts
            query = """
            LOAD CSV FROM 'file:///excerptsUFSC.csv'
            AS row
            MATCH  (t:Thesis{uri: row[0]})
            MERGE  (e:Exerpt{
                    uri: row[1],
                    text: row[2],
                    page: row[3],
                    lang: row[4]
                    })-[r:BFO_0000050]-(t);
            """
            graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
            results = graph.execute_query(query)
            graph.close()

            #Persons
            query = """
            LOAD CSV FROM 'file:///personsUFSC.csv'
            AS row
            MATCH  (e:Exerpt{uri: row[0]})
            MERGE  (p:Person{uri: row[1]})
            MERGE  (e)-[r:mentions]-(p);
            """
            graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
            results = graph.execute_query(query)
            graph.close()

            #GPEs
            query = """
            LOAD CSV FROM 'file:///gpesUFSC.csv'
            AS row
            MATCH  (e:Exerpt{uri: row[0]})
            MERGE  (p:Place{uri: row[1]})
            MERGE  (e)-[r:mentions]-(p);
            """
            graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
            results = graph.execute_query(query)
            graph.close()

            # Removendo os arquivos CSV após a importação
            os.remove('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\excerptsUFSC.csv')
            os.remove('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\personsUFSC.csv')
            os.remove('C:\\Users\\facordei\\.Neo4jDesktop\\relate-data\\dbmss\\dbms-792eabfc-5517-4220-ab29-30ef6ec3236e\\import\\gpesUFSC.csv')

            print(f"Total time: {pd.Timestamp.now() - start_time}")
            print(f"Partial time: {pd.Timestamp.now() - parcial_time}")
            # Atualizando a hora parcial
            parcial_time = pd.Timestamp.now()
    else:
        print(f"No PDF link found for {thesis_uri}")

    # gravando no grafo que a tese foi processada
    query = """
    MATCH  (t:Thesis{uri: \'""" + thesis_repository['Thesis'][i] +  """\'})
    SET t.processed = true
    SET t.repository = \'""" + str(repo) +  """\'
    """
    graph = GraphDatabase.driver(NEO4J_URL, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
    results = graph.execute_query(query)
    graph.close()


No PDF link found for imagens_invisiveis_de_africas_presentes__experiencias_das_populacoes_negras_no_cotidiano_da_cidade_de_florianopolis_19301940_
1 - Downloading and extracting: silencios_na_historia_e_disputas_entre_memorias_os_processos_de_reconhecimento_identitario_e_titulacao_territorial_das_comunidades_quilombolas from https://repositorio.ufsc.br/bitstream/handle/123456789/219203/PHST0710-T.pdf?sequence=-1&isAllowed=y


incorrect startxref pointer(1)


Loading triplas: silencios_na_historia_e_disputas_entre_memorias_os_processos_de_reconhecimento_identitario_e_titulacao_territorial_das_comunidades_quilombolas
Total time: 0 days 00:01:35.231187
Partial time: 0 days 00:01:35.231187
2 - Downloading and extracting: proposta_de_acoes_para_ingresso_e_permanencia_de_estudantes_indigenas_e_quilombolas_no_ensino_superior from https://repositorio.ufsc.br/bitstream/handle/123456789/254855/PPAU0301-D.pdf?sequence=-1&isAllowed=y


incorrect startxref pointer(1)


Loading triplas: proposta_de_acoes_para_ingresso_e_permanencia_de_estudantes_indigenas_e_quilombolas_no_ensino_superior
Total time: 0 days 00:02:12.510381
Partial time: 0 days 00:00:37.279194
No PDF link found for criminologia_e_racismo_
No PDF link found for mulheres_negras_soropositivas_e_as_interseccionalidades_entre_genero_classe_e_racaetnia
5 - Downloading and extracting: trajetorias_negras_e_racismos_memorias_da_convivencia_interracial_na_infancia from https://repositorio.ufsc.br/bitstream/handle/123456789/215891/PASO0525-D.pdf?sequence=-1&isAllowed=y


...

Erro ao ler o arquivo tese UFSC.pdf. 
Loading triplas: mathias_jose_dos_santos_identidade_escravidao_e_colonialidade_do_poder_18601875
Total time: 0 days 00:26:49.787560
Partial time: 0 days 00:00:24.434732
No PDF link found for de_herdeiro_da_fortuna_a_diplomata_aposentado_a_representacao_do_masculino_em_machado_de_assis
No PDF link found for o_crime_de_reduzir_pessoa_livre_a_escravidao_nas_casas_de_morada_da_justica_no_rio_grande_do_sul_18351874
No PDF link found for a_sinfonia_do_sagrado_em_castro_alves_deus_eros_e_mae_em_os_escravos
60 - Downloading and extracting: quanto_vale_um_escravo_hoje_a_vulnerabilidade_des_fiscalizada_os_rastros_da_necrobiopolitica_e_a_tentativa_de_controle_do_controle_dos_discursos_na_contemporaneidade from https://repositorio.ufsc.br/bitstream/handle/123456789/220487/PLLG0822-T.pdf?sequence=-1&isAllowed=y


incorrect startxref pointer(1)


Loading triplas: quanto_vale_um_escravo_hoje_a_vulnerabilidade_des_fiscalizada_os_rastros_da_necrobiopolitica_e_a_tentativa_de_controle_do_controle_dos_discursos_na_contemporaneidade
Total time: 0 days 00:28:24.437550
Partial time: 0 days 00:01:34.649990
No PDF link found for fugas_escravas_e_quilombos_na_ilha_de_santa_catarina_seculo_xix
No PDF link found for cativeiro_e_dependencia_na_fronteira_de_ocupacao_palmas_pr_18501888
63 - Downloading and extracting: os_sinais_da_escravidao_na_identidade_do_trabalhador_a_legislacao_neoescravista_da_caderneta_policial_de_trabalho_e_a_resistencia_dos_trabalhadores_do_comercio_no_rio_de_janeiro_da_1a_republica_df_anos_20 from https://repositorio.ufsc.br/bitstream/handle/123456789/192811/PDPC1384-T.pdf?sequence=-1&isAllowed=y


incorrect startxref pointer(1)


Erro ao ler o arquivo tese UFSC.pdf. 
Loading triplas: os_sinais_da_escravidao_na_identidade_do_trabalhador_a_legislacao_neoescravista_da_caderneta_policial_de_trabalho_e_a_resistencia_dos_trabalhadores_do_comercio_no_rio_de_janeiro_da_1a_republica_df_anos_20
Total time: 0 days 00:28:56.937630
Partial time: 0 days 00:00:32.500080
No PDF link found for papeis_da_escravidao_a_matricula_especial_de_escravos
No PDF link found for infancia_trabalho_escravo_e_direitos_sociais_o_brasil_e_os_direitos_humanos_em_questao


In [9]:
thesis_repository['title'][i]
#get_repository_link('Exterminate all the brutes colonialism and racism in Joseph Conrads heart of darkness')

'Exterminate all the brutes: colonialism and racism in Joseph Conrads heart of darkness'